# Assignment - Segmenting and Clustering Neighborhoods in Toronto

### Part 1

We use BeautifulSoup to scrape the list of postal codes of Canada from wikipedia (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M).

In [1]:
#make sure to have bs4, requests and lxml librairies installed

from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd
import numpy as np

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
PostalCodes_html = soup.find('table') #since there is a unique table in the page, no attribute is needed

In [5]:
PostalCodes_list = pd.read_html(str(PostalCodes_html))

In [6]:
PostalCodes_df = PostalCodes_list[0]
PostalCodes_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
PostalCodes_df = PostalCodes_df[PostalCodes_df['Borough'] != 'Not assigned']
PostalCodes_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
PostalCodes_df.groupby('Postal Code')
PostalCodes_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
PostalCodes_df['Neighborhood'] = np.where(PostalCodes_df['Neighborhood'] == 'Not assigned', PostalCodes_df['Borough'], PostalCodes_df['Neighborhood'])

C:\Users\2018t\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
PostalCodes_df.shape

(103, 3)

### Part 2

In [11]:
import geocoder

geoconder is not working for me, I will use the csv file provided.

In [12]:
    #even this classic example doesn't work
#url = 'https://maps.googleapis.com/maps/api/geocode/json'
#params = {'sensor': 'false', 'address': 'Mountain View, CA'}
#r = requests.get(url, params=params)
#results = r.json()['results']
#print(results)
#location = results[0]['geometry']['location']
#location['lat'], location['lng']

In [13]:
    #idem
#g = geocoder.google('Mountain View, CA')
#g.latlng
#print(g.latlng)

In [14]:
LatLng_df = pd.read_csv('Geospatial_Coordinates.csv')
LatLng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
neighborhoods = pd.merge(PostalCodes_df, LatLng_df, how='inner')
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Part 3

In [16]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from pandas.io.json import json_normalize

import folium # map rendering library

In [17]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
map_toronto

In [19]:
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [20]:
CLIENT_ID = '***' # your Foursquare ID
CLIENT_SECRET = '***' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AB313TZHYYA12USYENC01FLQNCOK4NE1A0SWVCFTQRK43DJ1
CLIENT_SECRET:YEBLQBB4IWJNYF1UP10VWQVN0JZH4GYSNPHBM13JAIOZTFNK


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
Toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
#re-run cell multiple time if an error occurs from 'groups'

In [24]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))
Toronto_venues[['Neighborhood','Venue']].groupby('Neighborhood').count()

There are 337 uniques categories.


,Venue
Neighborhood,
Agincourt,49
"Alderwood, Long Branch",25
"Bathurst Manor, Wilson Heights, Downsview North",29
Bayview Village,16
"Bedford Park, Lawrence Manor East",41
Berczy Park,100
"Birch Cliff, Cliffside West",13
"Brockton, Parkdale Village, Exhibition Place",100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",49


In [25]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopp

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 25

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Pizza Place,Sandwich Place,Bakery,Caribbean Restaurant,Japanese Restaurant,Sri Lankan Restaurant,Breakfast Spot,Latin American Restaurant,Bubble Tea Shop,Skating Rink,Lounge,Malay Restaurant,Seafood Restaurant,Cantonese Restaurant,Mediterranean Restaurant,Sushi Restaurant,Restaurant,Clothing Store,Coffee Shop,Motorcycle Shop,Pool Hall,Pool,Department Store
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Convenience Store,Pharmacy,Athletics & Sports,Donut Shop,Garden Center,Gas Station,Liquor Store,Moroccan Restaurant,Intersection,Coffee Shop,Shopping Mall,Trail,Sandwich Place,Pub,Grocery Store,Gym,Print Shop,Park,Entertainment Service,Falafel Restaurant,Doner Restaurant,Event Space,Ethiopian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Pizza Place,Coffee Shop,Mobile Phone Shop,Fried Chicken Joint,Sushi Restaurant,Supermarket,Ski Chalet,Ski Area,Mediterranean Restaurant,Shopping Mall,Middle Eastern Restaurant,Bridal Shop,Gas Station,Dog Run,Restaurant,Diner,Deli / Bodega,Park,Community Center,Pharmacy,Sandwich Place,Convenience Store,Ice Cream Shop,Trail
3,Bayview Village,Bank,Japanese Restaurant,Grocery Store,Gas Station,Café,Shopping Mall,Chinese Restaurant,Restaurant,Trail,Dog Run,Skating Rink,Park,Ethiopian Restaurant,Farmers Market,Field,Fast Food Restaurant,Donut Shop,Drugstore,Dry Cleaner,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Event Space
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Bank,Restaurant,Bridal Shop,Baby Store,Bagel Shop,Bakery,Butcher,Sushi Restaurant,Juice Bar,Pub,Indian Restaurant,Sports Club,Breakfast Spot,Liquor Store,Skating Rink,Café,Fast Food Restaurant,Thai Restaurant,Intersection,Cupcake Shop,Wings Joint,Pizza Place


In [27]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30]

array([1, 3, 1, 1, 2, 2, 2, 2, 2, 2, 3, 1, 1, 2, 2, 2, 1, 0, 2, 2, 2, 1,
       2, 1, 1, 2, 2, 2, 1, 2])

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged = Toronto_merged.dropna() #make we are able to assign a cluster to every Neighborhood
Toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Convenience Store,Pharmacy,Bus Stop,Shopping Mall,Discount Store,Fish & Chips Shop,Food & Drink Shop,Shop & Service,Laundry Service,Café,Supermarket,Caribbean Restaurant,Road,Tennis Court,Chinese Restaurant,Fast Food Restaurant,Cosmetics Shop,Coffee Shop,Pizza Place,Train Station,Grocery Store,Creperie,Eastern European Restaurant,Electronics Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Coffee Shop,Hockey Arena,Gym / Fitness Center,Sporting Goods Shop,French Restaurant,Café,Golf Course,Intersection,Men's Store,Pizza Place,Portuguese Restaurant,Curling Ice,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Cosmetics Shop,Entertainment Service,Event Space,Drugstore,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.0,Coffee Shop,Diner,Café,Park,Theater,Pub,Bakery,Breakfast Spot,Restaurant,Italian Restaurant,Sushi Restaurant,Gym / Fitness Center,Thai Restaurant,Performing Arts Venue,Gastropub,Indian Restaurant,Pool,Shoe Store,Pharmacy,Skating Rink,Pet Store,Cosmetics Shop,Burrito Place,Chocolate Shop,Sandwich Place
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Coffee Shop,Fast Food Restaurant,Fried Chicken Joint,Dessert Shop,Clothing Store,Vietnamese Restaurant,Restaurant,Furniture / Home Store,Sushi Restaurant,Bank,Discount Store,Korean Restaurant,Pet Store,Cheese Shop,Italian Restaurant,Sporting Goods Shop,Bowling Alley,Sandwich Place,Flea Market,Miscellaneous Shop,Fireworks Store,Seafood Restaurant,Boutique,Event Space,Gym / Fitness Center
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Coffee Shop,Park,Sushi Restaurant,Café,Ramen Restaurant,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Diner,Dance Studio,Cosmetics Shop,Yoga Studio,Men's Store,Pizza Place,Clothing Store,Restaurant,Bookstore,Burger Joint,Thai Restaurant,Bubble Tea Shop,Gay Bar,Gastropub,Ice Cream Shop,Discount Store,Gourmet Shop
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,3.0,Pharmacy,Convenience Store,Park,Playground,Skating Rink,Shopping Mall,Café,Bank,Golf Course,Grocery Store,Bakery,Deli / Bodega,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store,Cuban Restaurant,Creperie,Entertainment Service,Coworking Space,Event Space,Dry Cleaner,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Coffee Shop,Trail,Bakery,Arts & Crafts Store,Bank,Caribbean Restaurant,Supermarket,Restaurant,Greek Restaurant,Chinese Restaurant,Gym,Spa,Paper / Office Supplies Store,Business Service,Hobby Shop,Ethiopian Restaurant,Drugstore,Dry Cleaner,Field,Farmers Market,Eastern European Restaurant,Dumpling Restaurant,Event Space,Electronics Store
7,M3B,North York,Don Mills,43.745906,-79.352188,2.0,Restaurant,Japanese Restaurant,Coffee Shop,Gym,Bank,Supermarket,Burger Joint,Asian Restaurant,Pizza Place,Beer Store,Sushi Restaurant,Sporting Goods Shop,Italian Restaurant,Bike Shop,Jewelry Store,Mobile Phone Shop,Clothing Store,Intersection,Breakfast Spot,Middle Eastern Restaurant,Chinese Restaurant,Liquor Store,Shop & Service,Cafeteria,Café
8

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters